# Capstone Final Project

In [38]:
import urllib.request
wp = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page = wp.read()
from bs4 import BeautifulSoup
soup = BeautifulSoup(page)
first_table = soup.find("table", class_ = "wikitable sortable")

In [39]:

A=[]
B=[]
C=[]


for row in first_table.findAll("tr"):
    cells = row.findAll('td')
   
    if(len(cells)==3):
      # print(cells[0])
    
        borough=cells[1].find(text=True)
        neighborhood=cells[2].find(text=True)
        neighborhood=neighborhood.strip()
        borough=borough.strip()
        
        if(borough!="Not assigned"):
                
                    
                    A.append(cells[0].find(text=True))
                    B.append(borough)
                    if(neighborhood =="Not assigned"):
                        C.append(borough)
                    else:
                        C.append(neighborhood)

In [99]:
import pandas as pd

df=pd.DataFrame()
df['PostalCode']=A
df['Borough']=B
df['Neighbourhood']=C
result= df.groupby('PostalCode').agg(lambda x: ','.join(set(x))).reset_index()
result2=pd.read_csv("Geospatial_Coordinates.csv")
result3=result.merge(result2, on='PostalCode', how='left')



In [142]:
df_york = result3[result3['Borough'].str.contains("York")].reset_index(drop=True)


#### Setting up Foursquare API credentials

In [104]:
#sample code for credentialing, actual code hidden below

CLIENT_ID = '03DBHL5BGTMZWSW2SUQ411EVG0TJ2FD14FWWN5VJNMKLB50W' # your Foursquare ID
CLIENT_SECRET = 'GERGJBZQJIVLWR53K3GLUSKRPUSVR2QLQEFRUPXWUUJY2HHY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 03DBHL5BGTMZWSW2SUQ411EVG0TJ2FD14FWWN5VJNMKLB50W
CLIENT_SECRET:GERGJBZQJIVLWR53K3GLUSKRPUSVR2QLQEFRUPXWUUJY2HHY


In [105]:

LIMIT = 100 
radius = 500


In [106]:
# create function to extracts the category of the venue from json object 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [107]:
from pandas.io.json import json_normalize



In [109]:
# creating a function to repeat this with all neighbourhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue # appending values retrived from JSON
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [143]:
york_venues = getNearbyVenues(names=df_york['Neighbourhood'],
                                   latitudes=df_york['Latitude'],
                                   longitudes=df_york['Longitude']
                                  )


Hillcrest Village
Fairview,Oriole,Henry Farm
Bayview Village
Silver Hills,York Mills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill,Woodbine Gardens
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto
Bedford Park,Lawrence Manor East
Lawrence Heights,Lawrence Manor
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
Upwood Park,Maple Leaf Park,North Park
Mount Dennis,Silverthorn,Del Ray,Keelesdale
Runnymede,The Junction North
Humber Summit
Humberlea,Emery
Weston


In [113]:
#exploring a bit more

york_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",24,24,24,24,24,24
"CFB Toronto,Downsview East",4,4,4,4,4,4
Caledonia-Fairbanks,6,6,6,6,6,6
Don Mills North,5,5,5,5,5,5
Downsview Central,4,4,4,4,4,4
Downsview Northwest,4,4,4,4,4,4
Downsview West,4,4,4,4,4,4


In [144]:
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")
york_onehot['Neighbourhood'] = york_venues['Neighbourhood'] 
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
york_grouped = york_onehot.groupby('Neighbourhood').mean().reset_index()
print(york_grouped.shape)
york_grouped.head()

(33, 126)


,Neighbourhood,Accessories Store,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.000000,0.0
1,Bayview Village,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
2,"Bedford Park,Lawrence Manor East",0.0,0.00,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
3,"CFB Toronto,Downsview East",0.0,0.25,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
4,Caledonia-Fairbanks,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.0


In [146]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [147]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = york_grouped['Neighbourhood']



for ind in np.arange(york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Pizza Place,Sandwich Place,Bridal Shop,Diner
1,Bayview Village,Bank,Chinese Restaurant,Japanese Restaurant,Café,Yoga Studio
2,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Indian Restaurant,Pizza Place
3,"CFB Toronto,Downsview East",Airport,Park,Bus Stop,Electronics Store,Yoga Studio
4,Caledonia-Fairbanks,Park,Women's Store,Pharmacy,Market,Fast Food Restaurant


#### Finally to K-means clustering to see what neighbourhoods are close to each other in characteristics

######  Importing and applying Kmeans Algorithm

In [138]:
#importing Kmeans
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

york_grouped_clustering = york_grouped.drop('Neighbourhood', 1)
print(york_grouped_clustering)


# run k-means clustering
KMEANS = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated by algorithm
KMEANS.labels_[0:5] 

    Accessories Store  Airport  American Restaurant  Arcade  \
0            0.000000     0.00             0.000000    0.00   
1            0.000000     0.00             0.000000    0.00   
2            0.000000     0.00             0.041667    0.00   
3            0.000000     0.25             0.000000    0.00   
4            0.000000     0.00             0.000000    0.00   
5            0.000000     0.00             0.000000    0.00   
6            0.000000     0.00             0.000000    0.00   
7            0.000000     0.00             0.000000    0.00   
8            0.000000     0.00             0.000000    0.00   
9            0.000000     0.00             0.000000    0.00   
10           0.000000     0.00             0.015873    0.00   
11           0.000000     0.00             0.000000    0.00   
12           0.000000     0.00             0.000000    0.25   
13           0.000000     0.00             0.000000    0.00   
14           0.000000     0.00             0.000000    

array([0, 0, 0, 0, 0])

###### Create dataframe with cluster labels and above result

In [139]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', KMEANS.labels_)  #inserting Kmeans labels

york_merged = df_york         

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

york_merged.head()



,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Pool,Athletics & Sports,Mediterranean Restaurant,Dog Run,Golf Course
1,M2J,North York,"Fairview,Oriole,Henry Farm",43.778517,-79.346556,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Toy / Game Store
2,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Bank,Chinese Restaurant,Japanese Restaurant,Café,Yoga Studio
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN
4,M2M,North York,"Willowdale,Newtonbrook",43.789053,-79.408493,2.0,Gym,Yoga Studio,Empanada Restaurant,Comfort Food Restaurant,Construction & Landscaping


###### Creating map with clusters colored based on their cluster number

###### Checking two of the 5 clusters

In [140]:
york_merged.loc[york_merged['Cluster Labels'] == 3, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
33,York,3.0,Park,Yoga Studio,Empanada Restaurant,Construction & Landscaping,Convenience Store


In [141]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,0.0,Pool,Athletics & Sports,Mediterranean Restaurant,Dog Run,Golf Course
1,North York,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Toy / Game Store
2,North York,0.0,Bank,Chinese Restaurant,Japanese Restaurant,Café,Yoga Studio
5,North York,0.0,Restaurant,Ramen Restaurant,Coffee Shop,Japanese Restaurant,Sandwich Place
6,North York,0.0,Park,Bar,Bank,Yoga Studio,Dim Sum Restaurant
7,North York,0.0,Pizza Place,Coffee Shop,Pharmacy,Butcher,Comfort Food Restaurant
8,North York,0.0,Pool,Park,Food & Drink Shop,Fast Food Restaurant,Dog Run
9,North York,0.0,Gym / Fitness Center,Caribbean Restaurant,Basketball Court,Café,Japanese Restaurant
10,North York,0.0,Beer Store,Coffee Shop,Gym,Asian Restaurant,Chinese Restaurant
11,North York,0.0,Coffee Shop,Pizza Place,Sandwich Place,Bridal Shop,Diner
